In [ ]:
import os, glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import segmentation_models_pytorch as smp
import timm

# =========================
# Repro
# =========================
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

# =========================
# Dataset
# =========================
class BUSIDataset(Dataset):
    def __init__(self, df, resize=(256, 256)):
        self.df = df.reset_index(drop=True)
        self.resize = resize

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image"]
        mask_path = self.df.iloc[idx]["mask"]

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.resize is not None:
            img = img.resize(self.resize)
            mask = mask.resize(self.resize)

        img = np.array(img, dtype=np.float32) / 255.0
        mask = (np.array(mask, dtype=np.float32) / 255.0)
        mask = (mask > 0.5).astype(np.float32)

        img = torch.from_numpy(img).permute(2, 0, 1)
        mask = torch.from_numpy(mask).unsqueeze(0)
        return img, mask

# =========================
# DATASET PATHS (EDIT THIS)
# =========================
IMG_DIR  = "path/to/BUSI/Images"
MASK_DIR = "path/to/BUSI/Masks"

images = sorted(glob.glob(os.path.join(img_dir, "*.png")))
masks  = sorted(glob.glob(os.path.join(mask_dir, "*.png")))

assert len(images) == len(masks) and len(images) > 0, "Images/Masks count mismatch or empty!"

df = pd.DataFrame({"image": images, "mask": masks})
print(f"Total: {len(df)} image-mask pairs")



# Train/Val/Test = 70/15/15
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42, shuffle=True)
val_df,   test_df = train_test_split(temp_df, test_size=0.50, random_state=42, shuffle=True)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")



train_dataset = BUSIDataset(train_df)
val_dataset   = BUSIDataset(val_df)
test_dataset  = BUSIDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

# =========================
# TIMM Encoder Wrapper
# =========================
class TimmEncoderWrapper(nn.Module):
    def __init__(self, model_name, in_channels=3, depth=5, pretrained=True):
        super().__init__()
        self.model = timm.create_model(
            model_name,
            features_only=True,
            pretrained=pretrained,
            in_chans=in_channels
        )
        self._out_channels = self.model.feature_info.channels()[:depth]

    def forward(self, x):
        return self.model(x)[:len(self._out_channels)]

    @property
    def out_channels(self):
        return self._out_channels

# List of TIMM backbones to test
timm_encoders = [
    "swin_tiny_patch4_window7_224", "swin_base_patch4_window7_224",
    "pvt_v2_b0", "pvt_v2_b2","pvt_v2_b3",
    "vit_base_patch16_224", "vit_small_patch32_224"
]

# =========================
# Loss + Metrics
# =========================
def weighted_bce_dice_loss(pred, target):
    t = target.view(-1)
    num_pos = torch.clamp(t.sum(), min=0.)
    num_tot = t.numel()
    num_neg = num_tot - num_pos
    pos_weight = (num_neg / (num_pos + 1e-6)).to(target.device) if num_pos > 0 else torch.tensor(1.0, device=target.device)

    bce = F.binary_cross_entropy_with_logits(pred, target, pos_weight=pos_weight)
    p = torch.sigmoid(pred)
    intersection = (p * target).sum(dim=(1,2,3))
    denom = p.sum(dim=(1,2,3)) + target.sum(dim=(1,2,3)) + 1e-6
    dice = (2.0 * intersection + 1e-6) / denom
    dice_loss = 1.0 - dice.mean()
    return bce + dice_loss

@torch.no_grad()
def compute_metrics_from_logits(pred_logits, masks):
    probs = torch.sigmoid(pred_logits)
    preds = (probs > 0.5).float()
    TP = (preds * masks).sum().item()
    FP = (preds * (1 - masks)).sum().item()
    FN = ((1 - preds) * masks).sum().item()
    eps = 1e-7
    precision = TP / (TP + FP + eps)
    recall    = TP / (TP + FN + eps)
    f1        = 2 * precision * recall / (precision + recall + eps)
    iou       = TP / (TP + FP + FN + eps)
    dice      = (2 * TP) / (2 * TP + FP + FN + eps)
    return precision, recall, f1, iou, dice

# =========================
# Train / Eval
# =========================
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    loss_sum = 0.0
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device).float()
        optimizer.zero_grad(set_to_none=True)
        logits = model(imgs)
        loss = weighted_bce_dice_loss(logits, masks)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    return loss_sum / max(1, len(loader))

@torch.no_grad()
def eval_loss(model, loader, device):
    model.eval()
    loss_sum = 0.0
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device).float()
        logits = model(imgs)
        loss = weighted_bce_dice_loss(logits, masks)
        loss_sum += loss.item()
    return loss_sum / max(1, len(loader))

@torch.no_grad()
def evaluate_metrics(model, loader, device):
    model.eval()
    totals = {"precision":0.0, "recall":0.0, "f1":0.0, "iou":0.0, "dice":0.0}
    n = 0
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device).float()
        logits = model(imgs)
        p, r, f1, iou, d = compute_metrics_from_logits(logits, masks)
        totals["precision"] += p
        totals["recall"]    += r
        totals["f1"]        += f1
        totals["iou"]       += iou
        totals["dice"]      += d
        n += 1
    for k in totals:
        totals[k] /= max(1, n)
    return totals

# =========================
# Training loop
# =========================
def run_training(encoder_name, train_loader, val_loader, test_loader, device, epochs=20, lr=1e-4):
    encoder = TimmEncoderWrapper(encoder_name)
    model = smp.Unet(
        encoder=encoder,    # pass encoder object directly
        in_channels=3,
        classes=1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    print(f"\n=== Training {encoder_name.upper()} for {epochs} epochs ===")
    for epoch in range(1, epochs+1):
        tr_loss = train_one_epoch(model, train_loader, optimizer, device)
        va_loss = eval_loss(model, val_loader, device)
        print(f"Epoch {epoch:02d}/{epochs} | Train Loss: {tr_loss:.4f} | Val Loss: {va_loss:.4f}")

    test_metrics = evaluate_metrics(model, test_loader, device)
    return model, test_metrics


# =========================
# Run experiments
# =========================
device = "cuda" if torch.cuda.is_available() else "cpu"

results = []
for enc in timm_encoders:
    print(f"\n🚀 Training with encoder: {enc}")
    _, test_metrics = run_training(enc, train_loader, val_loader, test_loader, device, epochs=20)
    results.append({"Model": enc, **test_metrics})

df_results = pd.DataFrame(results)
print("\n📊 Final Comparison Results:")
print(df_results)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00:00:0100:01
Total: 1875 image-ma

model.safetensors:   0%|          | 0.00/181M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]


=== Training PVT_V2_B3 for 20 epochs ===
Epoch 01/20 | Train Loss: 1.2519 | Val Loss: 0.8787
Epoch 02/20 | Train Loss: 0.6784 | Val Loss: 0.5814
Epoch 03/20 | Train Loss: 0.4643 | Val Loss: 0.4864
Epoch 04/20 | Train Loss: 0.3590 | Val Loss: 0.3748
Epoch 05/20 | Train Loss: 0.3044 | Val Loss: 0.3671
Epoch 06/20 | Train Loss: 0.2662 | Val Loss: 0.3654
Epoch 07/20 | Train Loss: 0.2235 | Val Loss: 0.3786
Epoch 08/20 | Train Loss: 0.1973 | Val Loss: 0.3848
Epoch 09/20 | Train Loss: 0.1814 | Val Loss: 0.4635
Epoch 10/20 | Train Loss: 0.1828 | Val Loss: 0.4636
Epoch 11/20 | Train Loss: 0.1693 | Val Loss: 0.3421
Epoch 12/20 | Train Loss: 0.1600 | Val Loss: 0.5839
Epoch 13/20 | Train Loss: 0.2709 | Val Loss: 0.2966
Epoch 14/20 | Train Loss: 0.1779 | Val Loss: 0.3113
Epoch 15/20 | Train Loss: 0.1564 | Val Loss: 0.3238
Epoch 16/20 | Train Loss: 0.1364 | Val Loss: 0.3107
Epoch 17/20 | Train Loss: 0.1276 | Val Loss: 0.3381
Epoch 18/20 | Train Loss: 0.1192 | Val Loss: 0.3273
Epoch 19/20 | Train Lo